In [1]:
import pandas as pd
import re 

In [2]:
cfp=pd.read_csv("cfpb.csv",names=["complaintdate","product","subproduct"
,"issue","subissue","narrative","company_public_response","company","state"
,"zip","tags","companyconsent","via","send_to_company_date","company_response_consumer","timely","consumer_disputed","id"],skiprows=1)


In [3]:
cfp=cfp[(cfp['product']=='Mortgage')]

In [4]:
cfp=cfp[~cfp.narrative.isnull()]
cfp.shape

(39734, 18)

In [9]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
cfp["tokens"] = cfp["narrative"].apply(tokenizer.tokenize)
cfp.head()

,complaintdate,product,subproduct,issue,subissue,narrative,company_public_response,company,state,zip,tags,companyconsent,via,send_to_company_date,company_response_consumer,timely,consumer_disputed,id,tokens
25,11/18/2016,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,Started the refinance of home mortgage process...,NaN,AMERICAN NEIGHBORHOOD MORTGAGE,NJ,088XX,NaN,Consent provided,Web,11/21/2016,Closed with monetary relief,No,No,2216206,"[Started, the, refinance, of, home, mortgage, ..."
26,7/16/2015,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,"In XXXX, I and my ex-husband applied for a ref...",NaN,HSBC NORTH AMERICA HOLDINGS INC.,IL,625XX,NaN,Consent provided,Web,7/16/2015,Closed with explanation,Yes,No,1472017,"[In, XXXX, I, and, my, ex, husband, applied, f..."
29,4/27/2016,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,Mortgage was transferred to Nationstar as of X...,NaN,NATIONSTAR MORTGAGE,CA,954XX,NaN,Consent provided,Web,4/27/2016,Closed with explanation,Yes,Yes,1898476,"[Mortgage, was, transferred, to, Nationstar, a..."
69,6/30/2016,Mortgage,Conventional fixed mortgage,Credit decision / Underwriting,NaN,Need to move into a XXXX facility. Can no long...,NaN,"VANDERBILT MORTGAGE & FINANCE, INC",GA,300XX,Older American,Consent provided,Web,7/1/2016,Closed with explanation,Yes,No,1992309,"[Need, to, move, into, a, XXXX, facility, Can,..."
85,7/20/2015,Mortgage,FHA mortgage,"Loan servicing, payments, escrow account",NaN,I had an FHA loan at US Bank that was paid off...,Company chooses not to provide a public response,U.S. BANCORP,TN,370XX,NaN,Consent provided,Web,7/20/2015,Closed with explanation,Yes,No,1474887,"[I, had, an, FHA, loan, at, US, Bank, that, wa..."


In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer();

In [11]:
mortgage_dict=['refinance','borrower','mortgage','lender','reporting','refinanced','financed','finance']

In [12]:
def stemandlemma(narrative):
    
    preprocess_list = list()
    words = narrative.lower().split()
    #print(words)
    for w in words:
        if(w not in mortgage_dict):
            preprocess_list.append( lemmatizer.lemmatize(w) )
        else:
            preprocess_list.append(w)
    return " ".join(preprocess_list)

In [13]:
def removeNER(narrative):
    
    preprocess_list = list()
    words = narrative.lower().split()
    #print(words)
    for w in words:
        preprocess_list.append( stemmer.stem(w) )
    return preprocess_list

In [14]:
def preprocess_narrative( narrative ):
    
    preprocess_list = list()
    for text in narrative:
        letters_only = re.sub("[^a-zA-Z]", " ", text)
        letters_only = re.sub("XX","",letters_only)
        words = letters_only.lower().split()
        stops = set(stopwords.words("english"))
        meaningful_words = [w for w in words if not w in stops]
        #print(meaningful_words)
        #meaningful_words = [w for w in meaningful_words stemmer.stem(w)]
        preprocess_list.append(" ".join( meaningful_words ))
    return preprocess_list

In [15]:
import numpy as np

In [16]:
cfp['complaintdate'] = pd.to_datetime(cfp['complaintdate'])

In [17]:
cfp['newnarrative'] = preprocess_narrative( cfp.narrative )
cfp['newnarrative'] = cfp['newnarrative'].map(stemandlemma)

In [19]:
cfp["issue"].value_counts()

Loan servicing, payments, escrow account                                            14728
Loan modification,collection,foreclosure                                            10791
Application, originator, mortgage broker                                             3749
Trouble during payment process                                                       2898
Struggling to pay mortgage                                                           2401
Settlement process and costs                                                         2250
Credit decision / Underwriting                                                       1289
Applying for a mortgage or refinancing an existing mortgage                           752
Closing on a mortgage                                                                 624
Incorrect information on your report                                                  133
Applying for a mortgage                                                                62
Problem wi

In [24]:
ISSUE_LIST=cfp.loc[:,'issue']
ISSUE_LIST.head(20)

25                 Settlement process and costs
26     Application, originator, mortgage broker
29     Loan servicing, payments, escrow account
69               Credit decision / Underwriting
85     Loan servicing, payments, escrow account
95     Loan modification,collection,foreclosure
118    Loan servicing, payments, escrow account
161                Settlement process and costs
172    Loan modification,collection,foreclosure
175    Loan servicing, payments, escrow account
177    Loan servicing, payments, escrow account
201    Loan servicing, payments, escrow account
238                Settlement process and costs
254                Settlement process and costs
261    Loan servicing, payments, escrow account
278    Application, originator, mortgage broker
293    Loan servicing, payments, escrow account
302    Application, originator, mortgage broker
306    Loan modification,collection,foreclosure
322    Loan servicing, payments, escrow account
Name: issue, dtype: object

In [52]:
from collections import defaultdict
from collections import OrderedDict
def findMostUsedWords(cfp,issue,value,mydict):
    wordtokens=cfp.tokens[issue[ISSUE_LIST]=="value"]
    for words in wordtokens:
        for word in words:
            mydict[word]+=1
    return OrderedDict(sorted(mydict.items(),reverse=True,key=lambda item:item[1]))

In [53]:
for issue in ISSUE_LIST:
    mydict=defaultdict(int)
    print(issue,list(findMostUsedWords(cfp,issue,1,mydict).items())[:5])

TypeError: string indices must be integers